In [1]:
import numpy as np
import datetime as dt
import os
import pandas as pd
import math
import scipy
import requests
import warnings
import json
import h5py
from pyproj import Proj, CRS
import pysolid
import matplotlib.pyplot as plt
import rasterio as rio
from isce3.core import Ellipsoid as ellips

from osgeo import gdal, osr
import geopandas as gpd
import shapely.wkt as wkt
from shapely import geometry

from src.ALE_utils import oversample_slc, findCR
warnings.filterwarnings('ignore')

In [2]:
# Parameters for papermill
burst_id = 't107_227888_iw2'
date = '20221031'
snr_threshold = 10
solidtide = 'False'   #solid earth tide

In [3]:
pol = 'VV'
path_h5 = f'stack/{burst_id}/{date}/{burst_id}_{date}.h5'
fn = f'{burst_id}_{date}_VV.h5'

# Load the CSLC and necessary metadata
DATA_ROOT = 'science/SENTINEL1'
grid_path = f'{DATA_ROOT}/CSLC/grids'
metadata_path = f'{DATA_ROOT}/CSLC/metadata'
burstmetadata_path = f'{DATA_ROOT}/CSLC/metadata/processing_information/s1_burst_metadata'
id_path = f'{DATA_ROOT}/identification'

with h5py.File(path_h5,'r') as h5:
    cslc = h5[f'{grid_path}/{pol}'][:]
    xcoor = h5[f'{grid_path}/x_coordinates'][:]
    ycoor = h5[f'{grid_path}/y_coordinates'][:]
    dx = h5[f'{grid_path}/x_spacing'][()].astype(int)
    dy = h5[f'{grid_path}/y_spacing'][()].astype(int)
    epsg = h5[f'{grid_path}/projection'][()].astype(int)
    sensing_start = h5[f'{burstmetadata_path}/sensing_start'][()].astype(str)
    sensing_stop = h5[f'{burstmetadata_path}/sensing_stop'][()].astype(str)
    dims = h5[f'{burstmetadata_path}/shape'][:]
    bounding_polygon =h5[f'{id_path}/bounding_polygon'][()].astype(str) 
    orbit_direction = h5[f'{id_path}/orbit_pass_direction'][()].astype(str)
    center_lon, center_lat = h5[f'{burstmetadata_path}/center']
    
cslc_poly = wkt.loads(bounding_polygon)

In [4]:
path_ramp_h5 = f'stack/{burst_id}/{date}/{burst_id}_{date}_ramp.h5'

with h5py.File(path_ramp_h5,'r') as h5:
    ramp = h5[f'{grid_path}/{pol}'][:]     #reading geocoded ramp
    
cslc = cslc * np.conj(ramp)    #deramped geocoded CSLC

In [5]:
# Get the cslc date
date_ = dt.datetime.strptime(sensing_start.astype(str),'%Y-%m-%d %H:%M:%S.%f').strftime('%Y-%m-%d+%H\u0021%M')

# Download corner reflector data from NISAR based on the date of the CSLC product (Oklahoma)
# tide + plate motion are already corrected
res = requests.get(f'https://uavsar.jpl.nasa.gov/cgi-bin/corner-reflectors.pl?date={str(date_)}&project=nisar')
open('crdata.csv', 'wb').write(res.content)

# Read to pandas dataframe and rename columns
df = pd.read_csv('crdata.csv')
df.rename(columns={'Corner reflector ID':'ID'}, inplace=True)
df.rename(columns={'Latitude (deg)':'lat'}, inplace=True) 
df.rename(columns={'Longitude (deg)':'lon'}, inplace=True) 
df.rename(columns={'Azimuth (deg)':'azm'}, inplace=True)
df.rename(columns={'Tilt / Elevation angle (deg)':'tilt'}, inplace=True)
df.rename(columns={'Height above ellipsoid (m)':'hgt'}, inplace=True) 
df.rename(columns={'Side length (m)':'slen'}, inplace=True)

df.head()

,ID,lat,lon,hgt,azm,tilt,slen
0,N01K,35.591905,-98.932226,480.1540,359.5,13.68,2.8
1,N02K,35.586896,-99.267030,491.0222,183.5,15.73,2.8
2,N03K,35.588804,-98.935684,479.3191,180.5,14.53,2.8
3,N04K,35.583656,-99.267167,491.4049,0.5,17.90,2.8


In [6]:
if (solidtide == 'True' ):

    #solid earth tide correction with PySolid
    dateformat = '%Y-%m-%d %H:%M:%S.%f'  #date format of input azimuth time
    dt0 = dt.datetime.strptime(sensing_start,dateformat)
    dt1 = dt.datetime.strptime(sensing_stop,dateformat)
    step_sec = 5                        # sample spacing in time domain in seconds

    for idx, row in df.iterrows():

        llh = [np.deg2rad(row['lon']), np.deg2rad(row['lat']), row['hgt']]  #lon/lat/hgt

        _elp = ellips()
        xyz = _elp.lon_lat_to_xyz(llh) #xyz coordinate of CR

        # compute SET via pysolid
        (dt_out,
         tide_e,
         tide_n,
         tide_u) = pysolid.calc_solid_earth_tides_point(np.rad2deg(llh[1]), np.rad2deg(llh[0]), dt0, dt1,
                                                    step_sec=step_sec,
                                                    display=False,
                                                    verbose=False)

        tide_e = np.mean(tide_e[0:2])
        tide_n = np.mean(tide_n[0:2])
        tide_u = np.mean(tide_u[0:2])

        #updating lat,lon,hgt after SET correction
        xyz = [xyz[0]+tide_e, xyz[1]+tide_n, xyz[2]+tide_u]
        llh = _elp.xyz_to_lon_lat(xyz)  

        df.loc[idx,'lat'] = np.rad2deg(llh[1])
        df.loc[idx,'lon'] = np.rad2deg(llh[0])
        df.loc[idx,'hgt'] = llh[2]   

In [7]:
if 'lon' not in df.keys():
    raise SystemExit('No CRs found within burst, exit notebook')

In [8]:
#calculating the locations of CRs in SAR image
UTMx = []
UTMy = []
xloc = []
yloc = []
xloc_float = []
yloc_float = []
_in = []

for idx, row in df.iterrows():
    
    _Proj = Proj(CRS.from_epsg(epsg))
    _x, _y = _Proj(row['lon'], row['lat'],inverse=False)     #conversion of lat/lon of CRs to UTM coordinates
    
    #location of CRs in SLC image
    _xloc = int((_x-xcoor[0])/dx)    
    _yloc = int((_y-ycoor[0])/dy)
    
    UTMx.append(_x) 
    UTMy.append(_y)
    xloc.append(_xloc)
    yloc.append(_yloc)
    xloc_float.append((_x-xcoor[0])/dx)
    yloc_float.append((_y-ycoor[0])/dy)
    _in.append(cslc_poly.contains(geometry.Point(row['lon'], row['lat'])))
    
df['UTMx'] = UTMx
df['UTMy'] = UTMy
df['xloc'] = xloc
df['yloc'] = yloc
df['xloc_float'] = xloc_float
df['yloc_float'] = yloc_float
df['inPoly'] = _in

#checking whether CRs are in SLC coverage. Including only CRs within SLC image
df = df[df['inPoly']==True]
df.drop('inPoly', axis=1, inplace=True)
df = df.reset_index(drop=True)
df.head()

,ID,lat,lon,hgt,azm,tilt,slen,UTMx,UTMy,xloc,yloc,xloc_float,yloc_float
0,N01K,35.591905,-98.932226,480.1540,359.5,13.68,2.8,506139.631631,3.938688e+06,16099,1965,16099.426326,1965.664105
1,N02K,35.586896,-99.267030,491.0222,183.5,15.73,2.8,475808.319568,3.938164e+06,10033,2018,10033.163914,2018.139874
2,N03K,35.588804,-98.935684,479.3191,180.5,14.53,2.8,505826.604351,3.938344e+06,16036,2000,16036.820870,2000.071746
3,N04K,35.583656,-99.267167,491.4049,0.5,17.90,2.8,475794.948082,3.937804e+06,10030,2054,10030.489616,2054.079142


In [9]:
# #Displaying SLC image
# buffer = 50
# minX = df['xloc'].min() - buffer
# maxX = df['xloc'].max() + buffer
# minY = df['yloc'].min() - buffer
# maxY = df['yloc'].max() + buffer

# scale_ = 1.0
# exp_ = 0.15

# fig, ax = plt.subplots(figsize=(12, 3))
# cax=ax.imshow(scale_*(np.abs(cslc))**exp_, cmap='gray',interpolation=None, origin='upper')
# ax.set_xlim(minX,maxX)
# ax.set_ylim(minY,maxY)
# #ax.axis('off')

# for sl in pd.unique(df.slen):
#     xx = df.loc[df['slen']==sl]['xloc']
#     yy = df.loc[df['slen']==sl]['yloc']
#     ID = df.loc[df['slen']==sl]['ID']
    
#     if sl == 2.4384:
#         color='blue'
#     elif sl == 4.8:
#         color='red'
#     elif sl == 2.8:
#         color='yellow'
#     else:
#         color='green'
    
#     ax.scatter(xx,yy,color=color,marker="+",lw=1)
#     for _ID,_xx,_yy in zip(ID,xx,yy):
#         ax.annotate(_ID, (_xx, _yy), fontsize=10)

# ax.set_aspect(1)
# fig.savefig('S1_CSLC_CRs.png',dpi=300,bbox_inches='tight')

In [10]:
# selecting CRs according to orbit direction
if orbit_direction == 'Ascending':
    df_filter = df[(df['azm']>140) & (df['azm']<220)].reset_index(drop=True)
    #only west-looking CRs (for right-looking ascending)
else:     #Descending
    df_filter = df[~((df['azm']>140) & (df['azm']<220))].reset_index(drop=True)
    #only east-looking CRs (for right-looking descending)

df_filter = df_filter.loc[df_filter['slen']>0.8].reset_index(drop=True)   #excluding SWOT CRs (0.7 m as a side length)
df = None

In [11]:
# #removing shifted CRs
# #Period of CR damages
# #N02K: 20220410 ~ 20220621
# #N03K: 20220528 ~ 20220609

# if (int(date) >= 20220410) & (int(date) <= 20220621):     
#     df_filter = df_filter.loc[df_filter['ID']!='N02K'].reset_index(drop=True)

# if (int(date) >= 20220528) & (int(date) <= 20220913):
#     df_filter = df_filter.loc[df_filter['ID']!='N03K'].reset_index(drop=True)

# if df_filter.empty:
#     raise SystemExit('No CRs remaining, exit notebook')

# df_filter

In [12]:
# buffer = 3
# dB_cslc = 20*np.log10(np.abs(cslc))
# dB_ = []

# for ID, xoff, yoff in zip(df_filter['ID'],df_filter['xloc'],df_filter['yloc']):
#     minX = xoff - buffer
#     maxX = xoff + buffer
#     minY = yoff - buffer
#     maxY = yoff + buffer
    
#     dB_crop = dB_cslc[minY:maxY,minX:maxX]
#     dB_.append(np.mean(dB_crop))
    
# df_filter['dB'] = dB_

# dB_thresh = 36.    #threshold for dB at CRs
# df_filter = df_filter.loc[df_filter['dB']>dB_thresh].reset_index(drop=True)   #excluding CRs with low dB

# if df_filter.empty:
#     raise SystemExit('No CRs remaining, exit notebook')

# df_filter

In [13]:
# #Displaying SLC image
# buffer = 50
# minX = df_filter['xloc'].min() - buffer
# maxX = df_filter['xloc'].max() + buffer
# minY = df_filter['yloc'].min() - buffer
# maxY = df_filter['yloc'].max() + buffer

# scale_ = 1.0
# exp_ = 0.15

# fig, ax = plt.subplots(figsize=(12, 3))
# cax=ax.imshow(scale_*(np.abs(cslc))**exp_, cmap='gray',interpolation=None, origin='upper')
# ax.set_xlim(minX,maxX)
# ax.set_ylim(minY,maxY)

# for sl in pd.unique(df_filter.slen):
#     xx = df_filter.loc[df_filter['slen']==sl]['xloc']
#     yy = df_filter.loc[df_filter['slen']==sl]['yloc']
#     ID = df_filter.loc[df_filter['slen']==sl]['ID']
    
#     if sl == 2.4384:
#         color='blue'
#     elif sl == 4.8:
#         color='red'
#     elif sl == 2.8:
#         color='yellow'
#     else:
#         color='green'
    
#     ax.scatter(xx,yy,color=color,marker="+",lw=1)
#     for _ID,_xx,_yy in zip(ID,xx,yy):
#         ax.annotate(_ID, (_xx, _yy), fontsize=10)

# ax.set_aspect(1)
# fig.savefig('S1_CSLC_CRs_'+orbit_direction+'.png',dpi=300,bbox_inches='tight')

In [14]:
def get_snr_peak(img: np.ndarray, cutoff_percentile: float=3.0):
    '''
    Estimate the signal-to-noise ration (SNR) of the peak
    in the input image patch
    Parameter
    ---------
    img: numpy.ndarray
        SLC image patch to calculate the SNR
    cutout: float
        Cutout ratio of high and low part of the signal to cutoff
    Returns
    -------
    snr_peak_db: float
        SNR of the peak in decibel (db)
    '''

    power_arr = img.real ** 2 + img.imag ** 2

    # build up the mask array
    thres_low = np.nanpercentile(power_arr, cutoff_percentile)
    thres_high = np.nanpercentile(power_arr, 100 - cutoff_percentile)
    mask_threshold = np.logical_and(power_arr < thres_low,
                                    power_arr > thres_high)
    mask_invalid_pixel = np.logical_and(power_arr <= 0.0,
                                        np.isnan(power_arr))
    ma_power_arr = np.ma.masked_array(power_arr,
                                      mask=np.logical_and(mask_threshold,
                                                          mask_invalid_pixel))

    peak_power = np.nanmax(power_arr)
    mean_background_power = np.mean(ma_power_arr)

    snr_peak_db = np.log10(peak_power / mean_background_power) * 10.0

    return snr_peak_db

In [15]:
xpeak = []
ypeak = []
snr = []

for ID, xoff, yoff in zip(df_filter['ID'],df_filter['xloc'],df_filter['yloc']):
    # crop a patch of 10*10 with center at the calculated CR position
    pxbuff = 5
    pybuff = 5
    cropcslc = cslc[(yoff-pybuff):(yoff+pybuff),(xoff-pxbuff):(xoff+pxbuff)]
    _snr = get_snr_peak(cropcslc)

    # fig, ax = plt.subplots(figsize=(9, 6))
    # cax=ax.imshow(20*np.log10(np.abs(cropcslc)), cmap='gray',interpolation=None,
    #               extent=(xoff-pxbuff,xoff+pxbuff+1,yoff+pybuff+1,yoff-pybuff),origin='upper')
    # ax.set_aspect(1)
    # fig.colorbar(cax)

    # find the peak amplitude in the 10*10 patch
    yind,xind = np.unravel_index(np.argmax(np.abs(cropcslc), axis=None), cropcslc.shape)
    
    # give a warning if the peak and the calculated postion are too far
    dyind = yind-pybuff; dxind = xind-pxbuff
    dist = math.sqrt(dyind**2+dxind**2)
    if dist > 5.0:
        warnings.warn(f'the most bright pixel and the xloc is too far for CR {ID}')
    
    # crop a patch of 32*32 but with its center at the peak
    xbuff = 16
    ybuff = 16
    ycrop = np.arange(yoff+dyind-ybuff,yoff+dyind+ybuff)
    xcrop = np.arange(xoff+dxind-xbuff,xoff+dxind+xbuff)
    cropcslc = cslc[ycrop,:][:,xcrop]

    # fig, ax = plt.subplots(figsize=(9, 6))
    # cax=ax.imshow(20*np.log10(np.abs(cropcslc)), cmap='gray',interpolation=None,
    #               extent=(xoff+dxind-xbuff,xoff+dxind+xbuff+1,yoff+dyind+ybuff+1,yoff+dyind-ybuff),origin='upper')
    # fig.colorbar(cax)
    # ax.set_aspect(1)

    # oversample this 32*32 patch by 32
    ovsFactor = 32
    cropcslc_ovs,ycrop_ovs,xcrop_ovs = oversample_slc(cropcslc,sampling=ovsFactor,y=ycrop,x=xcrop)

    # fig, ax = plt.subplots(figsize=(9, 6))
    # cax=ax.imshow(20*np.log10(np.abs(cropcslc_ovs)), cmap='gray',interpolation=None,
    #               extent=(xoff+dxind-xbuff,xoff+dxind+xbuff+1,yoff+dyind+ybuff+1,yoff+dyind-ybuff),origin='upper')
    # fig.colorbar(cax)
    # ax.set_aspect(1)

    # find the peak amplitude again in a 2 x 2 patch, it correspond to 
    # (2*ovsFactor) x (2*ovsFactor) in oversampled slc
    yoff2 = int(cropcslc_ovs.shape[0]/2)
    xoff2 = int(cropcslc_ovs.shape[1]/2)
    cropcslc2 = cropcslc_ovs[yoff2-ovsFactor:yoff2+ovsFactor+1,
                           xoff2-ovsFactor:xoff2+ovsFactor+1]
    yind2,xind2 = np.unravel_index(np.argmax(abs(cropcslc2), axis=None), cropcslc2.shape)
    dyind2 = yind2-ovsFactor; dxind2 = xind2-ovsFactor

    # crop a patch of 3x3 oversampled patch with center at the peak
    cropcslc2 = cropcslc_ovs[yoff2+dyind2-1:yoff2+dyind2+2,xoff2+dxind2-1:xoff2+dxind2+2]
    ycrop2 = ycrop_ovs[yoff2+dyind2-1:yoff2+dyind2+2]
    xcrop2 = xcrop_ovs[xoff2+dxind2-1:xoff2+dxind2+2]
    xxcrop2,yycrop2 = np.meshgrid(xcrop2,ycrop2)
    xxcrop2_f = xxcrop2.flatten()
    yycrop2_f = yycrop2.flatten()
    cropcslc2_f = cropcslc2.flatten()

    # fig, ax = plt.subplots(figsize=(9, 6))
    # cax=ax.imshow(20*np.log10(np.abs(cropcslc2)), cmap='gray',interpolation=None,
    #               extent=[xcrop2[0],xcrop2[-1],ycrop2[-1],ycrop2[0]],origin='upper')
    # ax.set_aspect(1)
    # fig.colorbar(cax)

    # Check if pixel values in a patch are non-NaN
    valid = ~(np.isnan(cropcslc2_f))
    count_valid = np.count_nonzero(valid)

    if count_valid == 0:
        _ypeak, _xpeak = [np.nan, np.nan]

    else:
        _ypeak,_xpeak = findCR(np.abs(cropcslc2_f[valid]),yycrop2_f[valid],xxcrop2_f[valid],
                            x_bound=[xcrop2[0],xcrop2[-1]],y_bound=[ycrop2[0],ycrop2[-1]],method="para")

    xpeak.append(_xpeak)
    ypeak.append(_ypeak)
    snr.append(_snr)
    
df_filter['xloc_CR'] = xpeak
df_filter['yloc_CR'] = ypeak
df_filter['snr'] = snr

In [16]:
df_filter = df_filter.dropna()
df_filter = df_filter[df_filter.snr > snr_threshold]
df_filter

,ID,lat,lon,hgt,azm,tilt,slen,UTMx,UTMy,xloc,yloc,xloc_float,yloc_float,xloc_CR,yloc_CR,snr
0,N02K,35.586896,-99.267030,491.0222,183.5,15.73,2.8,475808.319568,3.938164e+06,10033,2018,10033.163914,2018.139874,10033.092993,2018.178879,15.981686
1,N03K,35.588804,-98.935684,479.3191,180.5,14.53,2.8,505826.604351,3.938344e+06,16036,2000,16036.820870,2000.071746,16036.858813,2000.085502,15.917980


In [17]:
#absloute geolocation error in range and azimuth after corrections
ALE_Rg = (df_filter['xloc_CR'] -  df_filter['xloc_float'])*dx
ALE_Az = (df_filter['yloc_CR'] - df_filter['yloc_float'])*np.abs(dy)

In [18]:
# #plotting ALE
# fig, ax = plt.subplots(figsize=(8,6))
# sc = ax.scatter(ALE_Rg, ALE_Az, s=200, c=df_filter['slen'], alpha=0.8, marker='o')
# ax.legend(*sc.legend_elements(),facecolor='lightgray')
# ax.get_legend().set_title('side length (m)')
# for ii, txt in enumerate(df_filter.iloc[:,0]):
#     ax.annotate(txt, (ALE_Rg[ii],ALE_Az[ii]), color='orange')   #putting IDs in each CR
# ax.grid(True)
# ax.set_xlim(-12,12)
# ax.set_ylim(-12,12)
# ax.axhline(0, color='black')
# ax.axvline(0, color='black')
# ax.set_title(f'Range: {np.round(np.nanmean(ALE_Rg), 3)} +/- {np.round(np.nanstd(ALE_Rg),3)} m, \
#     Azimuth: {np.round(np.nanmean(ALE_Az),3)}, +/- {np.round(np.nanstd(ALE_Az),3)} m')
# ax.set_xlabel('Easting error (m)')
# ax.set_ylabel('Northing error (m)')
# fig.suptitle(f'Absolute geolocation error for {date}')
# fig.savefig(f'ALE_CSLC_{date}.png',dpi=300,bbox_inches='tight')

In [19]:
ALE_Rg_Mean = np.round(np.nanmean(ALE_Rg),3)
ALE_Az_Mean = np.round(np.nanmean(ALE_Az),3)
ALE_Rg_Stdev = np.round(np.nanstd(ALE_Rg),3)
ALE_Az_Stdev = np.round(np.nanstd(ALE_Az),3)

summary = []
summary.append([date, ALE_Rg_Mean, ALE_Rg_Stdev, ALE_Az_Mean, ALE_Az_Stdev])
summary_df = pd.DataFrame(summary)
summary_df.to_csv(f'ALE.csv', mode='a', header=False)

summary = []

for ID, ALE_Rg_, ALE_Az_ in zip(df_filter['ID'],ALE_Rg,ALE_Az):
    summary.append([ID, ALE_Rg_, ALE_Az_])
summary_df = pd.DataFrame(summary)
summary_df.to_csv(f'ALE_ID.csv', mode='a', header=False)